# isc2021 
https://github.com/facebookresearch/isc2021

# Step 1


下載isc2021




In [88]:
!git clone https://github.com/facebookresearch/isc2021.git

fatal: destination path 'isc2021' already exists and is not an empty directory.


下載套件

In [89]:
try:
  import dedalus.public as de
  print("Dedalus already installed")
except:
  print("Dedalus not installed yet. Let's do it.")

# Step 1: Install FFTW
!apt-get install libfftw3-dev
!apt-get install libfftw3-mpi-dev

# Step 2: Set paths for Dedalus installation
import os
os.environ['MPI_INCLUDE_PATH'] = "/usr/lib/x86_64-linux-gnu/openmpi/include"
os.environ['MPI_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
os.environ['FFTW_INCLUDE_PATH'] = "/usr/include"
os.environ['FFTW_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"

# Step 3: Install Dedalus using pip
!pip3 install dedalus

# Step 4: Check Dedalus is importable
print()
print()
try:
  import dedalus.public as de
  print("Dedalus successfully installed :)")
except:
  print("Error installing Dedalus :(")
  raise

Dedalus already installed
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libfftw3-dev is already the newest version (3.3.7-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libfftw3-mpi-dev is already the newest version (3.3.7-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Dedalus successfully installed :)


執行作者測試

In [90]:
%cd isc2021
!ls
print("...")

%cd lear_gist-1.2
!make
%cd ..

/content/isc2021
baselines	    data    lear_gist-1.2  README.md	     setup.py
CODE_OF_CONDUCT.md  docs    LICENSE	   requirements.txt
CONTRIBUTING.md     images  list_files	   scripts
...
/content/isc2021/lear_gist-1.2
make: Nothing to be done for 'all'.
/content/isc2021


In [91]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 特徵提取
先只放10張images做實驗

* 特徵提取 - query

In [92]:
!python baselines/gist_baseline.py \
    --file_list list_files/subset_1_queries \
    --image_dir images/queries \
    --o data/subset_1_queries_gist.hdf5 \
    --nproc 2

args= Namespace(file_list='list_files/subset_1_queries', giststream_exec='lear_gist-1.2/compute_gist_stream', i0=0, i1=-1, image_dir='images/queries', n_train_pca=10000, nproc=2, o='data/subset_1_queries_gist.hdf5', pca_dim=256, pca_file='', pca_white=0.0, train_pca=False, transpose=-1)
reading image names from list_files/subset_1_queries
hardware_image_description: model name : Intel(R) Xeon(R) CPU @ 2.20GHz, 2 cores
  found 10 images
computing features
processing 4/5 images/queries/Q01114.jpg
image_description_time: 0.01610 s per image
writing descriptors to data/subset_1_queries_gist.hdf5


* 特徵提取 - references

In [93]:
!python baselines/gist_baseline.py \
    --file_list list_files/subset_1_references \
    --image_dir images/references \
    --o data/subset_1_references_gist.hdf5 \
    --nproc 2

args= Namespace(file_list='list_files/subset_1_references', giststream_exec='lear_gist-1.2/compute_gist_stream', i0=0, i1=-1, image_dir='images/references', n_train_pca=10000, nproc=2, o='data/subset_1_references_gist.hdf5', pca_dim=256, pca_file='', pca_white=0.0, train_pca=False, transpose=-1)
reading image names from list_files/subset_1_references
hardware_image_description: model name : Intel(R) Xeon(R) CPU @ 2.20GHz, 2 cores
  found 10 images
computing features
processing 4/5 images/references/R021968.jpg
image_description_time: 0.02331 s per image
writing descriptors to data/subset_1_references_gist.hdf5


Evaluation

In [94]:
!python scripts/compute_metrics.py \
    --query_descs data/subset_1_queries_gist.hdf5 \
    --db_descs data/subset_1_references_gist.hdf5 \
    --gt_filepath list_files/subset_1_ground_truth.csv \
    --track2 \
    --max_dim 1000

Track 2 running matching of 10 queries in 10 database (960D descriptors), max_results=500000.
Evaluating 100 predictions (4991 GT matches)
Average Precision: 0.00095
Recall at P90    : 0.00060
Threshold at P90 : -0.259161
Recall at rank 1:  0.00100
Recall at rank 10: 0.00200


In [95]:
# By adding option --write_predictions data/predictions_subset_1_gist.csv to the script call above, 
# the predictions will be written out during the metrics computation

!python scripts/compute_metrics.py \
    --query_descs data/subset_1_queries_gist.hdf5 \
    --db_descs data/subset_1_references_gist.hdf5 \
    --gt_filepath list_files/subset_1_ground_truth.csv \
    --track2 \
    --max_dim 1000 \
    --write_predictions data/predictions_subset_1_gist.csv

Track 2 running matching of 10 queries in 10 database (960D descriptors), max_results=500000.
writing predictions to data/predictions_subset_1_gist.csv
Evaluating 100 predictions (4991 GT matches)
Average Precision: 0.00095
Recall at P90    : 0.00060
Threshold at P90 : -0.259161
Recall at rank 1:  0.00100
Recall at rank 10: 0.00200


In [96]:
!python scripts/compute_metrics.py \
    --preds_filepath data/predictions_subset_1_gist.csv \
    --gt_filepath list_files/subset_1_ground_truth.csv

Track 1 results of 100 predictions (4991 GT matches)
Average Precision: 0.00095
Recall at P90    : 0.00060
Threshold at P90 : -0.259161
Recall at rank 1:  0.00100
Recall at rank 10: 0.00200


In [97]:
!sort -n -r -t , -k 3 data/predictions_subset_1_gist.csv

query_id,reference_id,score
Q00911,R044290,-0.067817
Q00748,R007851,-0.119448
Q00814,R032607,-0.259161
Q00846,R007851,-0.268971
Q00748,R032607,-0.304911
Q00061,R014120,-0.308655
Q01094,R026396,-0.310027
Q00846,R032607,-0.313044
Q00911,R021968,-0.322693
Q00911,R033260,-0.340907
Q00061,R032607,-0.344489
Q00906,R014120,-0.358589
Q00846,R044290,-0.363175
Q00748,R021968,-0.370004
Q00846,R021968,-0.371518
Q00911,R032607,-0.390635
Q00748,R044290,-0.390966
Q01094,R014120,-0.394493
Q00061,R021968,-0.396236
Q01094,R044290,-0.399052
Q01114,R044290,-0.399519
Q01094,R032607,-0.400202
Q00846,R014120,-0.402414
Q00061,R007851,-0.411225
Q00061,R003142,-0.414866
Q00748,R014120,-0.415114
Q00911,R007851,-0.423515
Q00061,R044290,-0.430151
Q00911,R014120,-0.443364
Q00814,R044290,-0.444091
Q00911,R044970,-0.458363
Q01114,R021968,-0.459355
Q00846,R033260,-0.463421
Q01094,R021968,-0.466323
Q01114,R033260,-0.476734
Q00814,R007851,-0.486914
Q00338,R033260,-0.491402
Q00748,R044970,-0.494966
Q00748,R033260,-0.4980

In [98]:
# Evaluation with the official script
!python scripts/convert_track2_format.py \
    --query_descs data/subset_1_queries_gist.hdf5 \
    --db_descs data/subset_1_references_gist.hdf5 \
    --o data/subset_1_descriptors.hdf5

warning, dimension 960 is larger than allowed for track 2
writing data/subset_1_descriptors.hdf5


In [99]:
!python eval_metrics.py \
        data/predictions_subset_1_gist.csv \
        list_files/subset_1_ground_truth.csv

python3: can't open file 'eval_metrics.py': [Errno 2] No such file or directory


# Step 2

## 特徵提取

訓練 PCA

In [107]:
!python baselines/gist_baseline.py  \
      --pca_dim 30 \
      --file_list list_files/train \
      --image_dir images/train \
      --nproc 2 \
      --pca_file data/pca_gist.vt \
      --n_train_pca 101 \
      --train_pca

args= Namespace(file_list='list_files/train', giststream_exec='lear_gist-1.2/compute_gist_stream', i0=0, i1=-1, image_dir='images/train', n_train_pca=101, nproc=2, o='/tmp/desc.hdf5', pca_dim=30, pca_file='data/pca_gist.vt', pca_white=0.0, train_pca=True, transpose=-1)
reading image names from list_files/train
hardware_image_description: model name : Intel(R) Xeon(R) CPU @ 2.20GHz, 2 cores
  found 101 images
subsampled 101 vectors
computing features
processing 50/51 images/train/T000066.jpg
image_description_time: 0.01476 s per image
Train PCA 960 -> 30
Storing PCA to data/pca_gist.vt


運行特徵提取

In [109]:
!python baselines/gist_baseline.py \
    --file_list list_files/subset_1_queries \
    --image_dir images/queries \
    --o data/subset_1_queries_gist_pca.hdf5 \
    --pca_file data/pca_gist.vt \
    --nproc 2

args= Namespace(file_list='list_files/subset_1_queries', giststream_exec='lear_gist-1.2/compute_gist_stream', i0=0, i1=-1, image_dir='images/queries', n_train_pca=10000, nproc=2, o='data/subset_1_queries_gist_pca.hdf5', pca_dim=256, pca_file='data/pca_gist.vt', pca_white=0.0, train_pca=False, transpose=-1)
reading image names from list_files/subset_1_queries
hardware_image_description: model name : Intel(R) Xeon(R) CPU @ 2.20GHz, 2 cores
  found 10 images
computing features
processing 4/5 images/queries/Q01114.jpg
image_description_time: 0.01552 s per image
Load PCA matrix data/pca_gist.vt
Apply PCA 960 -> 30
writing descriptors to data/subset_1_queries_gist_pca.hdf5


In [110]:
!python baselines/gist_baseline.py \
    --file_list list_files/subset_1_references \
    --image_dir images/references \
    --o data/subset_1_references_gist_pca.hdf5 \
    --pca_file data/pca_gist.vt \
    --nproc 2

args= Namespace(file_list='list_files/subset_1_references', giststream_exec='lear_gist-1.2/compute_gist_stream', i0=0, i1=-1, image_dir='images/references', n_train_pca=10000, nproc=2, o='data/subset_1_references_gist_pca.hdf5', pca_dim=256, pca_file='data/pca_gist.vt', pca_white=0.0, train_pca=False, transpose=-1)
reading image names from list_files/subset_1_references
hardware_image_description: model name : Intel(R) Xeon(R) CPU @ 2.20GHz, 2 cores
  found 10 images
computing features
processing 4/5 images/references/R032607.jpg
image_description_time: 0.01981 s per image
Load PCA matrix data/pca_gist.vt
Apply PCA 960 -> 30
writing descriptors to data/subset_1_references_gist_pca.hdf5


Evalute

In [111]:
!python scripts/compute_metrics.py \
    --query_descs data/subset_1_queries_gist_pca.hdf5 \
    --db_descs data/subset_1_references_gist_pca.hdf5 \
    --gt_filepath list_files/subset_1_ground_truth.csv \
    --track2

Track 2 running matching of 10 queries in 10 database (30D descriptors), max_results=500000.
Evaluating 100 predictions (4991 GT matches)
Average Precision: 0.00093
Recall at P90    : 0.00040
Threshold at P90 : -0.0238483
Recall at rank 1:  0.00100
Recall at rank 10: 0.00200
